# Building the Asset Universe

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from openbb import obb
import financedatabase as fd
from financetoolkit import Toolkit
from utils.api_keys import API_KEYS

## Defining OpenBB Data Providers

In [6]:

# OBB Data Providers list:
#-------------------------#
obb_data_providers = [
    "alphavantage",
    "benzinga",
    "biztoc",
    "fmp",
    "fred",
    "nasdaq",
    "polygon",
    "tiingo",
    "yfinance"
]
#-------------------------#
# OBB interval list:
obb_intervals = []
    
#defining the function to get data from OBB
def get_obb_data(ticker, provider, start_date=None, end_date=None, interval=None):
    if provider not in obb_data_providers:
        print(f"Error: provider should be in {obb_data_providers}")
        return None
    try:
        data = obb.equity.price.historical(
            ticker, 
            provider=provider, 
            start_date=start_date, 
            end_date=end_date, 
            interval=interval
        ).to_df()
        return data
    except Exception as e:
        print(f"Error fetching data from {provider}: {e}")
        return None


Testing `OpenBB`to import AAPL data

In [7]:
apple_data = obb.equity.price.historical("AAPL", provider="yfinance").to_df()

## Testing Finance DataBase Package

In [8]:
# Load FMP API key from API_KEYS dictionary
fmp_api_key = API_KEYS["fmp"]

### FD Equities

In [ ]:
# Initialize the Equities database
equities = fd.Equities()

# Build Asset Universe based on sector and market cap
# Sector: Information Technology (IT)
# Market Cap: focus on Mid Cap stocks
asset_universe_equities = equities.select(
    sector="Information Technology",
    market_cap=["Mid Cap"],
)

In [42]:
# Show number of assets 
asset_universe_equities.groupby("exchange").size().sort_values(ascending=False).head(25)

exchange
FRA    236
NMS    122
PNK    109
SHZ     71
NYQ     63
STU     61
SHH     55
LSE     53
MEX     44
MUN     39
TAI     33
JPX     30
BER     27
GER     27
DUS     24
ASE     21
VIE     20
HKG     14
TOR     13
TLO     13
HAM     12
SAO     11
NSE     10
HAN      9
BSE      8
dtype: int64

Let's pick only top 25 countries from above:

In [43]:
# TOP 25 exchanges
asset_universe_exchanges = asset_universe_equities.groupby("exchange").size().sort_values(ascending=False).head(25).index.tolist()
asset_universe_exchanges

['FRA',
 'NMS',
 'PNK',
 'SHZ',
 'NYQ',
 'STU',
 'SHH',
 'LSE',
 'MEX',
 'MUN',
 'TAI',
 'JPX',
 'BER',
 'GER',
 'DUS',
 'ASE',
 'VIE',
 'HKG',
 'TOR',
 'TLO',
 'HAM',
 'SAO',
 'NSE',
 'HAN',
 'BSE']

In [44]:
asset_universe_equities = asset_universe_equities[asset_universe_equities["exchange"].isin(asset_universe_exchanges)]
asset_universe_equities

,name,summary,currency,sector,industry_group,industry,exchange,market,country,state,city,zipcode,website,market_cap,isin,cusip,figi,composite_figi,shareclass_figi
symbol,,,,,,,,,,,,,,,,,,,
000021.SZ,"Shenzhen Kaifa Technology Co., Ltd.","Shenzhen Kaifa Technology Co., Ltd. provides e...",CNY,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals",SHZ,Shenzhen Stock Exchange,China,NaN,Shenzhen,518035,http://www.kaifa.cn,Mid Cap,NaN,NaN,NaN,NaN,NaN
000050.SZ,"Tianma Microelectronics Co., Ltd.","Tianma Microelectronics Co., Ltd. designs, man...",CNY,Information Technology,Technology Hardware & Equipment,"Electronic Equipment, Instruments & Components",SHZ,Shenzhen Stock Exchange,China,NaN,Shenzhen,518052,http://www.tianma.cn,Mid Cap,NaN,NaN,NaN,NaN,NaN
000062.SZ,"Shenzhen Huaqiang Industry Co., Ltd.","Shenzhen Huaqiang Industry Co., Ltd. operates ...",CNY,Information Technology,Technology Hardware & Equipment,"Electronic Equipment, Instruments & Components",SHZ,Shenzhen Stock Exchange,China,NaN,Shenzhen,NaN,http://www.szhq000062.com,Mid Cap,NaN,NaN,NaN,NaN,NaN
000066.SZ,"China Greatwall Technology Group Co., Ltd.","China Greatwall Technology Group Co., Ltd. pro...",CNY,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals",SHZ,Shenzhen Stock Exchange,China,NaN,Shenzhen,NaN,http://www.greatwall.cn,Mid Cap,NaN,NaN,NaN,NaN,NaN
000547.SZ,"Addsino Co., Ltd.","Addsino Co., Ltd., together with its subsidiar...",CNY,Information Technology,Technology Hardware & Equipment,Communications Equipment,SHZ,Shenzhen Stock Exchange,China,NaN,Beijing,100195,http://www.casic-addsino.com,Mid Cap,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZOE.MU,TIS Inc.,TIS Inc. provides information technology (IT) ...,EUR,Information Technology,Software & Services,IT Services,MUN,Munich Stock Exchange,Japan,NJ,Tokyo,160-0023,http://www.tis.com,Mid Cap,NaN,NaN,NaN,NaN,NaN
ZOE.SG,TIS Inc.,TIS Inc. provides information technology (IT) ...,EUR,Information Technology,Software & Services,IT Services,STU,Stuttgart Stock Exchange,Japan,NJ,Tokyo,160-0023,http://www.tis.com,Mid Cap,NaN,NaN,NaN,NaN,NaN
ZOE.TI,TIS Inc.,TIS Inc. provides information technology (IT) ...,NaN,Information Technology,Software & Services,IT Services,TLO,EuroTLX,Japan,NJ,Tokyo,160-0023,http://www.tis.com,Mid Cap,NaN,NaN,NaN,NaN,NaN


List echanges:

In [46]:
# Show number of assets by market
display(asset_universe_equities.groupby("market").size().sort_values(ascending=False))
# Print the number of assets in the universe
print(f"Number of equities in the universe: {len(asset_universe_equities)}")

market
Frankfurt Stock Exchange                 236
NASDAQ Global Select                     122
OTC Bulletin Board                       109
Shenzhen Stock Exchange                   71
New York Stock Exchange                   63
Stuttgart Stock Exchange                  61
Shanghai Stock Exchange                   55
London Stock Exchange (international)     53
Mexico Stock Exchange                     44
Munich Stock Exchange                     39
Taiwan Stock Exchange                     33
Tokyo Stock Exchange                      30
XETRA                                     27
Berlin Stock Exchange                     27
Dusseldorf Stock Exchange                 24
NYSE MKT                                  21
Vienna Stock Exchange                     20
Hong Kong Stock Exchange                  14
EuroTLX                                   13
TSX Toronto Exchange                      13
Hamburg Stock Exchange                    12
Bovespa Soma                              11
Nat

Number of equities in the universe: 1125


### FD ETFs

In [47]:
etfs = fd.ETFs()
asset_universe_etfs = etfs.select(category_group="Information Technology")
# Show number of ETFs by market
display(asset_universe_etfs.groupby("exchange").size().sort_values(ascending=False))

exchange
BER    148
LSE    140
FRA    127
MUN    125
DUS     97
PCX     93
EBS     82
GER     67
MIL     66
MEX     61
TOR     52
HAM     48
PAR     29
NMS     26
AMS     25
BTS     23
NGM     22
HAN     12
KSC     11
ASX      9
MCX      7
TAI      6
PNK      4
NEO      3
NIM      3
JPX      3
SES      2
TLV      2
BUD      1
SET      1
TWO      1
dtype: int64

In [ ]:
# Select only the top 25 exchanges
asset_universe_etfs = asset_universe_etfs[asset_universe_etfs["exchange"].isin(asset_universe_exchanges)]
# Show number of ETFs by market
display(asset_universe_etfs.groupby("exchange").size().sort_values(ascending=False))
# Print the number of ETFs in the universe
print(f"Number of ETFs in the universe: {len(asset_universe_etfs)}")


exchange
BER    148
LSE    140
FRA    127
MUN    125
DUS     97
GER     67
MEX     61
TOR     52
HAM     48
NMS     26
HAN     12
TAI      6
PNK      4
JPX      3
dtype: int64

Number of ETFs in the universe: 916


In [49]:
asset_universe_etfs

,name,currency,summary,category_group,category,family,exchange
symbol,,,,,,,
00753L.TW,CTBC China 50 2X,TWD,The CTBC China 50 2X ETF aims to provide two t...,Information Technology,NaN,CTBC Securities Investment Trust,TAI
00757.TW,UPAMC NYSE FANG+ ETF,TWD,UPAMC NYSE FANG+ ETF aims to replicate the per...,Information Technology,NaN,Yuanta Securities Investment Trust,TAI
00770.TW,Cathay S&amp;P North Amrcn Tchnlgy Sctr ETF,TWD,Cathay S&P North Amrcn Tchnlgy Sctr ETF aims t...,Information Technology,NaN,Cathay Securities Investment Trust,TAI
008201.TW,W.I.S.E. - SSE 50 China Tracker,TWD,The investment seeks to provide the investment...,Information Technology,NaN,BOCI-Prudential Asset Management,TAI
00851.TW,Taishin SG Global AI and Robotics Select ETF,TWD,Taishin SG Global AI and Robotics Select ETF a...,Information Technology,NaN,Taishin Securities Investment Trust,TAI
...,...,...,...,...,...,...,...
ZPDT.BE,SPDR S+P US TECH.S.S.UETF,EUR,SPDR S+P US TECH.S.S.UETF is an exchange-trade...,Information Technology,Developed Markets,State Street Global Advisors,BER
ZPDT.DE,SPDR S&amp;P U.S. Technology Select Sector UCI...,EUR,The objective of the SPDR S&P U.S. Technology ...,Information Technology,Developed Markets,State Street Global Advisors,GER
ZPDT.DU,SPDR S+P US TECH.S.S.UETF,EUR,SPDR S+P US TECH.S.S.UETF is an exchange-trade...,Information Technology,Developed Markets,State Street Global Advisors,DUS


### FD Funds

In [51]:
# Initialize Funds database
funds = fd.Funds()
asset_universe_funds = funds.select(category_group="Information Technology")
# Pick only the top 25 exchanges
asset_universe_funds = asset_universe_funds[asset_universe_funds["exchange"].isin(asset_universe_exchanges)]    
# Show number of funds by market
display(asset_universe_funds.groupby("exchange").size().sort_values(ascending=False))
# Print the number of funds in the universe
print(f"Number of funds in the universe: {len(asset_universe_funds)}")

exchange
STU    40
FRA     9
VIE     6
LSE     4
TOR     3
BER     1
TAI     1
dtype: int64

Number of funds in the universe: 64


In [52]:
# display funds
asset_universe_funds

,name,currency,summary,category_group,category,family,exchange
symbol,,,,,,,
0P00000COJ.F,Allianz Informationstechnologie A EUR,EUR,Allianz Informationstechnologie A EUR is a tec...,Information Technology,Technology,Allianz Global Investors,FRA
0P00000CV8.F,DWS Telemedia Typ O ND,EUR,DWS Telemedia Typ O ND is a technology and med...,Information Technology,Technology,DWS Investment,FRA
0P00000GDW.L,Herald Worldwide Technology Fund Class A,GBP,Herald Worldwide Technology Fund Class A is a ...,Information Technology,Technology,Herald Investment Management Limited,LSE
0P00000MQ6.F,BNP Paribas Technologies Europe Classic,EUR,NaN,Information Technology,Technology,BNP Paribas Asset Management,FRA
0P00000OXY.F,Amundi Internetaktien,EUR,Amundi Internetaktien is a fund managed by Amu...,Information Technology,Technology,Amundi Asset Management,FRA
...,...,...,...,...,...,...,...
XC4M.SG,Fidelity Funds SICAV ? Global T,EUR,Fidelity Funds SICAV ? Global Technology Fund ...,Information Technology,Technology,NaN,STU
XDWT.SG,Xtrackers MSCI World Informatio,EUR,Xtrackers MSCI World Information Technology UC...,Information Technology,Technology,NaN,STU
XQ1Z.SG,FRANK-TECHNOLOGY-A ACCEUR,EUR,NaN,Information Technology,Technology,NaN,STU


### FD Cryptos

In [57]:
# Initialize the Crypto database
cryptos = fd.Cryptos()
eth_cryptos = cryptos.select(
    cryptocurrency="ETH"
)
cryptos_toolkit = eth_cryptos.to_toolkit(
    api_key=fmp_api_key,
    start_date="2020-01-01"
)
cryptos_toolkit.get_historical_data(period="daily")

Obtaining historical data:   0%|          | 0/6 [00:00<?, ?it/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Obtaining historical data: 100%|██████████| 6/6 [00:00<00:00,  9.16it/s]


The following tickers acquired historical data from FinancialModelingPrep: ETH-USD, SPY
The following tickers acquired historical data from YahooFinance: ETH-GBP, ETH-BTC, ETH-EUR, ETH-CAD


Open                                                  \
             ETH-BTC   ETH-CAD   ETH-EUR   ETH-GBP ETH-USD Benchmark   
2020-01-01  115.5363  115.5363  115.5363  115.5363  128.54   322.035   
2020-01-02  116.6047  116.6047  116.6047  116.6047  130.14    323.54   
2020-01-03  114.0503  114.0503  114.0503  114.0503  126.81    321.16   
2020-01-04  120.2281  120.2281  120.2281  120.2281  134.17  320.9367   
2020-01-05  121.0378  121.0378  121.0378  121.0378  134.14  320.7133   
...              ...       ...       ...       ...     ...       ...   
2025-04-17 1406.8057 1406.8057 1406.8057 1406.8057  1577.3    527.64   
2025-04-18 1384.8763 1384.8763 1384.8763 1384.8763 1583.86  528.7725   
2025-04-19 1391.4351 1391.4351 1391.4351 1391.4351 1588.61   529.905   
2025-04-20 1397.2145 1397.2145 1397.2145 1397.2145 1612.97  531.0375   
2025-04-21 1386.5444 1386.5444 1386.5444 1386.5444 1587.42    532.17   

                High                                ... Excess Volatility  \
             ETH-BTC   ETH-CAD   ETH-EUR   ETH-GBP  ...           ETH-EUR   
2020-01-01  118.7992  118.7992  118.7992  118.7992  ...            0.0492   
2020-01-02  116.6047  116.6047  116.6047  116.6047  ...            0.0492   
2020-01-03  120.5736  120.5736  120.5736  120.5736  ...            0.0492   
2020-01-04  121.9166  121.9166  121.9166  121.9166  ...            0.0492   
2020-01-05  124.9252  124.9252  124.9252  124.9252  ...            0.0492   
...              ...       ...       ...       ...  ...               ...   
2025-04-17 1418.1149 1418.1149 1418.1149 1418.1149  ...            0.0492   
2025-04-18 1422.9803 1422.9803 1422.9803 1422.9803  ...            0.0492   
2025-04-19 1406.4008 1406.4008 1406.4008 1406.4008  ...            0.0492   
2025-04-20 1432.4592 1432.4592 1432.4592 1432.4592  ...            0.0492   
2025-04-21 1437.1598 1437.1598 1437.1598 1437.1598  ...            0.0492   

                                     Cumulative Return                  \
           ETH-GBP ETH-USD Benchmark           ETH-BTC ETH-CAD ETH-EUR   
2020-01-01  0.0492  0.0502    0.0198               1.0     1.0     1.0   
2020-01-02  0.0492  0.0502    0.0198            0.9782  0.9782  0.9782   
2020-01-03  0.0492  0.0502    0.0198            1.0312  1.0312  1.0312   
2020-01-04  0.0492  0.0502    0.0198            1.0381  1.0381  1.0381   
2020-01-05  0.0492  0.0502    0.0198            1.0473  1.0473  1.0473   
...            ...     ...       ...               ...     ...     ...   
2025-04-17  0.0492  0.0502    0.0198            11.879  11.879  11.879   
2025-04-18  0.0492  0.0502    0.0198           11.9357 11.9357 11.9357   
2025-04-19  0.0492  0.0502    0.0198           11.9838 11.9838 11.9838   
2025-04-20  0.0492  0.0502    0.0198           12.1648 12.1648 12.1648   
2025-04-21  0.0492  0.0502    0.0198           11.6989 11.6989 11.6989   

                                      
           ETH-GBP ETH-USD Benchmark  
2020-01-01     1.0     1.0       NaN  
2020-01-02  0.9782   0.974       1.0  
2020-01-03  1.0312  1.0306    0.9924  
2020-01-04  1.0381  1.0303    0.9937  
2020-01-05  1.0473  1.0393    0.9949  
...            ...     ...       ...  
2025-04-17  11.879 12.1655     1.753  
2025-04-18 11.9357 12.2019    1.7388  
2025-04-19 11.9838 12.3886    1.7246  
2025-04-20 12.1648 12.1923    1.7104  
2025-04-21 11.6989 12.5503    1.6963  

[1938 rows x 72 columns]